In [29]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

In [30]:
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(api_key=groq_api_key, model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B03B19B800>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B03B193290>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [31]:
#HF for embeddings
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')

In [32]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [33]:
import bs4

In [48]:
loader = WebBaseLoader(
    web_path=("https://www.ibm.com/think/topics/artificial-intelligence"),
    bs_kwargs = dict(
        parse_only = bs4.SoupStrainer()
        )
    )
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.ibm.com/think/topics/artificial-intelligence', 'title': 'What Is Artificial Intelligence (AI)? | IBM', 'description': 'Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision-making, creativity and autonomy.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\nWhat Is Artificial Intelligence (AI)? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCost of a Data Breach Report 2025\nAI is becoming a security risk. Are you covered?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                        \n\n\n\n  \n    What is artificial intelligence (AI)?\n\n\n\n\n\n\n    \n\n\n                    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    \n\n\n\n  \n    9 August 2024\n\n\n\n\n\n\n    

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectordb = Chroma.from_documents(documents = splits, embedding = embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001B03B375520>, search_kwargs={})

In [50]:
# prompt template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


In [51]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}")
    ]
)

In [52]:
qa_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever,qa_chain)

In [58]:
response = rag_chain.invoke({"input": "Wwho is Bill Gates?"})
response

{'input': 'Wwho is Bill Gates?',
 'context': [Document(id='5a0db214-b98d-4b50-ad51-6758e3536053', metadata={'title': 'What Is Artificial Intelligence (AI)? | IBM', 'source': 'https://www.ibm.com/think/topics/artificial-intelligence', 'language': 'en', 'description': 'Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision-making, creativity and autonomy.'}, page_content='Meet Granite\n\n\n\n\n\n\n\n\n                Report\n            \n\n                IBM is named a Leader in Data Science & Machine Learning\n            \nLearn why IBM has been recognized as a Leader in the 2025 Gartner® Magic Quadrant™ for Data Science and Machine Learning Platforms.\n\nRead the report\n\n\n\n\n\n\n\n\n                Video\n            \n\n                IBM AI Academy\n            \nLed by top IBM thought leaders, the curriculum is designed to help business leaders gain the knowledge needed to prioritize

In [56]:
response = rag_chain.invoke({"input": "what is Deep Learning?"})
response

{'input': 'what is Deep Learning?',
 'context': [Document(id='80c47459-dcb1-49dc-bd17-223d85e4a1c3', metadata={'language': 'en', 'title': 'What Is Artificial Intelligence (AI)? | IBM', 'source': 'https://www.ibm.com/think/topics/artificial-intelligence', 'description': 'Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision-making, creativity and autonomy.'}, page_content='Deep learning is a subset of machine learning that uses multilayered neural networks, called deep neural networks, that more closely simulate the complex decision-making power of the human brain.\nDeep neural networks include an input layer, at least three but usually hundreds of hidden layers, and an output layer, unlike neural networks used in classic machine learning models, which usually have only one or two hidden layers.\nThese multiple layers enable unsupervised learning: they can automate the extraction of features fr

In [59]:
response = rag_chain.invoke({"input": "what it enables us to do?"})
response

{'input': 'what it enables us to do?',
 'context': [Document(id='ecd27261-448a-425a-82e6-420e481e36c2', metadata={'description': 'Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision-making, creativity and autonomy.', 'language': 'en', 'title': 'What Is Artificial Intelligence (AI)? | IBM', 'source': 'https://www.ibm.com/think/topics/artificial-intelligence'}, page_content='Explore our 2025 guide to AI agents\n\n\n\n\n\n\n\n\n\n\r\n        Benefits of AI\xa0\r\n    \n\n\n\nAI offers numerous benefits across various industries and applications. Some of the most commonly cited benefits include:\nAutomation of repetitive tasks.More and faster insight from data.\nEnhanced decision-making.\nFewer human errors.24x7 availability.Reduced physical risks.\n\n\nAutomation of repetitive tasks\n\n\nAI can automate routine, repetitive and often tedious tasks including digital tasks such as data collection,

## Adding Chat History

In [60]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [61]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

In [62]:
contextualize_q_system_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ]
)

In [64]:
history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_system_prompt)
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001B03B375520>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessag

In [65]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ]
)

In [66]:
qa_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,qa_chain)

In [67]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "What is Deep Learning?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"Deep learning is a subset of machine learning that uses multilayered neural networks to simulate the human brain's decision-making power. It enables the automation of feature extraction from large, unlabeled data sets and makes its own predictions about the data. This type of learning is well-suited for natural language processing, computer vision, and tasks involving complex pattern recognition."

In [69]:
conversational_rag_chain.invoke(
    {"input": "what it enables us to do?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Deep learning enables us to perform tasks such as identifying complex patterns and relationships in large amounts of data, simulating human learning and comprehension, and making detailed recommendations to users and experts. It also allows for the creation of devices and applications that can see, identify objects, understand and respond to human language, and learn from new information and experience.'